In [7]:
from unsupervised.SVD_np import SVD
from numpy import array

In [4]:
A = array([
 [1,2,3,4,5,6,7,8,9,10],
 [11,12,13,14,15,16,17,18,19,20],
 [21,22,23,24,25,26,27,28,29,30]])

In [8]:
my_svd = SVD(n_components = 2)
my_svd.fit(A)
y = my_svd.transform(A)

In [9]:
y

array([[-18.52157747,   6.47697214],
       [-49.81310011,   1.91182038],
       [-81.10462276,  -2.65333138]])

In [ ]:
my_svd = SVD(n_components = 1)
my_svd.fit(input_matrix)
my_svd.transform(input_matrix)